In [21]:
import pandas as pd
import scipy.stats as sp

call_df = pd.read_csv('user_calls.csv')
status_df = pd.read_csv('user_logs/user_status.csv')

In [36]:
group_df = call_df.groupby(by=['user_id'])['call_type'].count()
group_df = pd.DataFrame(group_df)
group_df['user_id'] = group_df.index
group_df = group_df.merge(status_df, on=['user_id'])
group_df['status'] = group_df['status'] == 'repaid'

In [42]:
# clear positive association between number of calls and loan status
sp.pearsonr(group_df['call_type'], group_df['status'])

(0.49147750318399142, 1.028787562126901e-25)